In [ ]:
import pandas as pd
import os

In [ ]:
#Reading the data into two files depending on what year it's from
annual_merged_data = {
    '2011-2012': pd.read_csv('data/raw/2011-2012_cognitive.csv'), 
    '2013-2014': pd.read_csv('data/raw/2013-2014_cognitive.csv')
}
# annual_merged_data

In [ ]:
if 'processed' not in os.listdir('data/'):
    os.mkdir('data/processed')

In [ ]:
#Iterates through each file in raw data folder
for file in os.listdir('data/raw/'):
    #Gets the year from each file by using the split function and indexing to position 0 in the list
    year = file.split('_')[0]
    #Gets data header from the file by indexing to position 1 in the list
    questionnaire = file.split('_')[1]
    #First part is checking if the year of the file is either 2011-2012 or 2013-2014
    #Second part is making sure that the file is not cognitive.csv, because this is what our model is trying to predict
    if year not in annual_merged_data or questionnaire == 'cognitive.csv':
        #It will move onto the next file by going through the for loop again
        continue
    #Reading file into variable
    data_snippet_df = pd.read_csv(f'data/raw/{file}')
    #Adding the file to it's respective year Key in the dictionary. It's making each file name a column. Combining all csv data
    annual_merged_data[year] = annual_merged_data[year].merge(data_snippet_df, how = 'left', on = 'SEQN')
#Summary: Goes through all files in raw data folder, it updates the data frame columms in the dictionary by participant number for both 2011-2012 and 2013-2014 tables. It adds all data, not just cognitive questionnaire data.
#Combines all data 

In [ ]:
#Organizes data into a table. Puts all column headers in one line
merged_nhanes_df = pd.concat(annual_merged_data.values(), axis=0)
merged_nhanes_df

In [ ]:
#Cleaning up the data to disclude empty values and people who didn't fully complete the cognitive tests
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df['CFDCCS'] == 1] #CERAD Completion Status (Gets rid of ppl who didn't complete all 4 recalls)
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df['CFDCST1'].notna()] #CERAD Score Trial 1
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df['CFDCSR'].notna()] #CERAD Score Delayed Recall
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df['CFDDS'].notna()]  #Digit Symbol Score
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df['CFDAST'].notna()] #Animal Fluency Score Total

In [ ]:
#Proportion missing: number of empty data cells and dividing by number of people (rows)
prop_missing = merged_nhanes_df.isnull().sum() / len(merged_nhanes_df)
#Updating dataframe so it only includes columns with proportion missing less than 0.1 - Cleaning it up
merged_nhanes_df = merged_nhanes_df[merged_nhanes_df.columns[(prop_missing < 0.1)]]

In [ ]:
#Converting to a csv after all cleaned up
merged_nhanes_df.to_csv('data/processed/merged_nhanes.csv', index=False)